# カテゴリ変数分析 v2

**目的**: カテゴリカル特徴量の分析とTarget Encodingの検討

**このノートブックで行うこと**:
1. カーディナリティ分類
2. Target Encoding効果の予測
3. カテゴリ別ターゲット統計量の確認

## セットアップ

In [ ]:
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# プロジェクトルートをパスに追加
project_root = Path().resolve().parent.parent
sys.path.insert(0, str(project_root))
sys.path.insert(0, str(project_root / "04_src"))
os.chdir(project_root)

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns

# フォント設定
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
try:
    import japanize_matplotlib
    japanize_matplotlib.japanize()
except ImportError:
    plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meiryo', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# テスト済み関数のimport
from eda.categorical import (
    classify_cardinality,
    calculate_target_encoding_potential,
    get_category_target_stats
)

print("✓ セットアップ完了")

## 1. データ読み込み

In [ ]:
# データ読み込み（DataLoaderパターン）
from src.data.loader import DataLoader
from src.utils.config import load_config

data_config = load_config("data")
loader = DataLoader(data_config)
train = loader.load_train()

print(f"訓練データ: {train.shape}")
train.head()

## 2. カテゴリカラムの抽出

In [ ]:
# カテゴリカラム（String型 + Int64で離散値）を抽出
string_cols = [col for col in train.columns if train[col].dtype == pl.String]

# Int64で離散値と思われるもの（ユニーク数が行数の50%以下）
int_cols = [col for col in train.columns if train[col].dtype == pl.Int64]
discrete_int_cols = [
    col for col in int_cols 
    if train[col].n_unique() < len(train) * 0.5
]

# ターゲット変数を除外
target_col = "money_room"
categorical_cols = [col for col in string_cols + discrete_int_cols if col != target_col]

print(f"カテゴリカラム数: {len(categorical_cols)}")
print(f"  String型: {len(string_cols)}")
print(f"  Int64離散: {len(discrete_int_cols)}")

## 3. カーディナリティ分類

In [ ]:
# カーディナリティで分類（テスト済み関数）
cardinality_result = classify_cardinality(train, categorical_cols)

print("[カーディナリティ分類]")
print(f"低カーディナリティ（<10）: {len(cardinality_result['low'])}件")
print(f"中カーディナリティ（10-50）: {len(cardinality_result['medium'])}件")
print(f"高カーディナリティ（>50）: {len(cardinality_result['high'])}件")

print("\n[低カーディナリティ - 上位10件]")
for col, n_unique in cardinality_result['low'][:10]:
    print(f"  {col:40s}: {n_unique:3d}件")

print("\n[中カーディナリティ - 上位10件]")
for col, n_unique in cardinality_result['medium'][:10]:
    print(f"  {col:40s}: {n_unique:3d}件")

print("\n[高カーディナリティ - 上位10件]")
for col, n_unique in cardinality_result['high'][:10]:
    print(f"  {col:40s}: {n_unique:3d}件")

## 4. Target Encoding効果の予測

In [ ]:
# 低〜中カーディナリティでTarget Encoding効果を予測
te_candidates = cardinality_result['low'] + cardinality_result['medium']

te_potentials = []

for col, n_unique in te_candidates:
    # テスト済み関数でTarget Encoding効果を計算
    potential = calculate_target_encoding_potential(train, col, target_col)
    te_potentials.append((col, n_unique, potential))

# ポテンシャル降順でソート
te_potentials.sort(key=lambda x: x[2], reverse=True)

print("[Target Encoding効果予測 - 上位20件]")
print(f"{'カラム名':40s} | ユニーク数 | 標準偏差")
print("-" * 70)
for col, n_unique, potential in te_potentials[:20]:
    print(f"{col:40s} | {n_unique:10d} | {potential:10.2f}")

## 5. カテゴリ別ターゲット統計量（上位カテゴリ）

In [ ]:
# Target Encoding効果が高い上位3カラムで詳細分析
top_3_cols = [col for col, _, _ in te_potentials[:3]]

for col in top_3_cols:
    # カテゴリ別統計量を取得（テスト済み関数）
    stats = get_category_target_stats(train, col, target_col, top_n=15)
    
    print(f"\n[{col} - カテゴリ別ターゲット統計（上位15件）]")
    print(stats)
    
    # 可視化
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))
    
    # 平均値のバープロット
    categories = stats[col].to_list()
    means = stats['mean_target'].to_list()
    
    ax1.barh(range(len(categories)), means)
    ax1.set_yticks(range(len(categories)))
    ax1.set_yticklabels([str(c)[:30] for c in categories])
    ax1.set_xlabel('平均ターゲット値')
    ax1.set_title(f'{col} - カテゴリ別平均（上位15件）')
    ax1.axvline(x=train[target_col].mean(), color='r', linestyle='--', label='全体平均')
    ax1.legend()
    
    # 件数のバープロット
    counts = stats['count'].to_list()
    
    ax2.barh(range(len(categories)), counts)
    ax2.set_yticks(range(len(categories)))
    ax2.set_yticklabels([str(c)[:30] for c in categories])
    ax2.set_xlabel('件数')
    ax2.set_title(f'{col} - カテゴリ別件数（上位15件）')
    
    plt.tight_layout()
    plt.show()

## 6. まとめ

### カーディナリティ
- **低カーディナリティ**: X件 → One-Hot Encodingが適用可能
- **中カーディナリティ**: Y件 → Target Encodingが有効
- **高カーディナリティ**: Z件 → Frequency EncodingやLeave-One-Out Encodingを検討

### Target Encoding候補
- **効果が高い特徴量**: [リスト]
- **注意点**: 
  - 過学習を防ぐためクロスバリデーションでエンコード
  - テストデータに未知カテゴリがある場合の対策が必要

### 次のステップ
- [ ] 地理空間分析（05_geospatial_analysis.ipynb）
- [ ] カテゴリエンコーディングの実装
- [ ] 特徴量エンジニアリング